In [1]:
from nnsight import LanguageModel
from nnsight.envoy import Envoy
from nnsight.util import WrapperModule
import torch
from typing import List, Tuple, Dict, Any

from transformers.utils import fx as tfx
import torch.fx as fx
from torch.fx import replace_pattern

/share/u/caden/.conda/envs/interp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = LanguageModel("gpt2", device_map="auto", dispatch=True)

with model.trace("a"):
    shape_stuff = model.transformer.h[3].attn.output.shape

attention = model._model.transformer.h[3].attn
attention_envoy = model._envoy.transformer.h[3].attn

KeyboardInterrupt: 

In [ ]:
wrapper_module = WrapperModule()
wrapper_name = 'output_wrapper'

setattr(attention, wrapper_name, wrapper_module)

attention

GPT2Attention(
  (c_attn): Conv1D()
  (c_proj): Conv1D()
  (attn_dropout): Dropout(p=0.1, inplace=False)
  (resid_dropout): Dropout(p=0.1, inplace=False)
  (output_wrapper): WrapperModule()
)

In [ ]:
def custom_backend(gm: torch.fx.GraphModule, example_inputs: List[torch.Tensor]):
    print("Custom backend called with FX graph:")


    # for node in gm.graph.nodes:    
        
        # if node.op == 'output' and node.name == "output":
            
        #     with gm.graph.inserting_before(node):
        #         wrapper_args = node.args
        #         wrapper_kwargs = node.kwargs
        #         wrapper_node = gm.graph.call_module(wrapper_name, args=wrapper_args, kwargs=wrapper_kwargs)

        #         node.replace_all_uses_with(wrapper_node)

    gm.recompile()

    gm.graph.print_tabular()
    
    return gm.forward

torch._dynamo.reset()


opt_model = torch.compile(attention, backend=custom_backend, dynamic=True)
gm = opt_model(attention_envoy._fake_inputs[0][0][0])

Custom backend called with FX graph:
opcode       name       target     args            kwargs
-----------  ---------  ---------  --------------  -----------------------
placeholder  s0         s0         ()              {}
placeholder  l_tensor_  L_tensor_  ()              {}
call_method  to         to         (l_tensor_, 0)  {'non_blocking': False}
output       output     output     ((to,),)        {}
Custom backend called with FX graph:
opcode         name             target                                                    args                                    kwargs
-------------  ---------------  --------------------------------------------------------  --------------------------------------  --------
placeholder    l_stack0_0_0_    L_stack0_0_0_                                             ()                                      {}
placeholder    l_module_nf      L_module_nf                                               ()                                      {}
placeholder   